In [1]:
from scipy.sparse import coo_matrix
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import matplotlib.pyplot as plt
import ast

In [80]:
!unzip preprocessed_data-20230421T050432Z-001 

unzip:  cannot find or open preprocessed_data-20230421T050432Z-001, preprocessed_data-20230421T050432Z-001.zip or preprocessed_data-20230421T050432Z-001.ZIP.


In [2]:


animes_data_path = "preprocessed_animes.csv"
profiles_data_path = "preprocessed_profiles.csv"
reviews_data_path = "preprocessed_reviews.csv"
animes_data = pd.read_csv(animes_data_path)
profiles_data = pd.read_csv(profiles_data_path)
reviews_data = pd.read_csv(reviews_data_path, engine='python', sep=',', error_bad_lines=False)

import pickle 
vit_50_dict = {}
with open("compressed_vit_50_mapping_embeddings.pickle", "rb") as f:
  vit_50_dict = pickle.load(f)
q1 = reviews_data['anime_uid'].unique().tolist()
q2 = [x for x in vit_50_dict.keys()]
s1=set(q1)
s2 = set(q2)
s3 = s1-s2 
l3 = list(s3)

animes = animes_data.copy()
profiles = profiles_data.copy()
reviews = reviews_data.copy()
for x in l3:
  reviews.drop(reviews[reviews['anime_uid'] == x].index, inplace = True)




<ipython-input-2-a3bbfe32d236>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  reviews_data = pd.read_csv(reviews_data_path, engine='python', sep=',', error_bad_lines=False)


In [227]:
reviews_data

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,"first things first. my ""reviews"" system is exp...",8,"{'Sound': '10', 'Overall': '8', 'Animation': '...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,let me start off by saying that made in abyss ...,10,"{'Story': '10', 'Animation': '10', 'Sound': '1...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,"art 9/10: it is great, especially the actions ...",7,"{'Story': '7', 'Sound': '8', 'Overall': '7', '...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,taking place 1 yr from where season 1 trailed ...,9,"{'Overall': '9', 'Story': '9', 'Enjoyment': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,kyoto animations greatest strength is being ab...,10,"{'Story': '10', 'Sound': '9', 'Overall': '10',...",https://myanimelist.net/reviews.php?id=291149
...,...,...,...,...,...,...,...
84145,72705,Paulwillson26,2593,the kara no kyoukai or garden of sinners films...,8,"{'Story': '7', 'Animation': '10', 'Sound': '10...",https://myanimelist.net/reviews.php?id=72705
84146,146535,iHitokage,2593,you know nothing jon snow.... this is how i fe...,10,"{'Story': '10', 'Animation': '10', 'Sound': '1...",https://myanimelist.net/reviews.php?id=146535
84147,13126,Onegai,2593,so i finally decided to watch the kara no kyou...,7,"{'Overall': '7', 'Story': '6', 'Enjoyment': '6...",https://myanimelist.net/reviews.php?id=13126
84148,127899,Murasa22,2593,this review is based on all the movies of kara...,10,"{'Story': '9', 'Enjoyment': '9', 'Animation': ...",https://myanimelist.net/reviews.php?id=127899


In [3]:
reviews[reviews.profile=='DesolatePsyche']

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,"first things first. my ""reviews"" system is exp...",8,"{'Sound': '10', 'Overall': '8', 'Animation': '...",https://myanimelist.net/reviews.php?id=255938
1377,255793,DesolatePsyche,12403,"first things first. my ""reviews"" system is exp...",6,"{'Story': '6', 'Overall': '6', 'Animation': '9...",https://myanimelist.net/reviews.php?id=255793
2088,298207,DesolatePsyche,35073,"first things first. my ""reviews"" system is exp...",6,"{'Story': '6', 'Enjoyment': '6', 'Animation': ...",https://myanimelist.net/reviews.php?id=298207
2427,255795,DesolatePsyche,23225,"first things first. my ""reviews"" system is exp...",7,"{'Overall': '7', 'Enjoyment': '6', 'Animation'...",https://myanimelist.net/reviews.php?id=255795
2879,291256,DesolatePsyche,2035,"first things first. my ""reviews"" system is exp...",8,"{'Story': '9', 'Sound': '8', 'Animation': '7',...",https://myanimelist.net/reviews.php?id=291256
...,...,...,...,...,...,...,...
81403,264298,DesolatePsyche,3269,"first things first. my ""reviews"" system is exp...",6,"{'Animation': '6', 'Sound': '5', 'Story': '6',...",https://myanimelist.net/reviews.php?id=264298
81456,254406,DesolatePsyche,4549,"first things first. my ""reviews"" system is exp...",7,"{'Story': '7', 'Overall': '7', 'Sound': '9', '...",https://myanimelist.net/reviews.php?id=254406
81985,256154,DesolatePsyche,30014,"first things first. my ""reviews"" system is exp...",7,"{'Story': '7', 'Overall': '7', 'Sound': '6', '...",https://myanimelist.net/reviews.php?id=256154
82717,261743,DesolatePsyche,33589,"first things first. my ""reviews"" system is exp...",7,"{'Story': '7', 'Overall': '7', 'Enjoyment': '6...",https://myanimelist.net/reviews.php?id=261743


In [4]:

np.random.seed(0)
unique_AnimeID = reviews['anime_uid'].unique()
unique_users = reviews['profile'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_users:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_AnimeID:
    movie_old2new_id_dict[i] = j
    j += 1
    


In [5]:
movie_old2new_id_dict.keys()

dict_keys([34096, 34599, 28891, 2904, 4181, 16664, 4672, 5114, 30276, 1253, 1425, 7588, 20767, 22673, 6862, 451, 9563, 30364, 33280, 36369, 27633, 8609, 36214, 5680, 31637, 552, 38145, 34944, 1689, 35363, 32863, 322, 38958, 7645, 468, 97, 39940, 4472, 19319, 1571, 38770, 5005, 18397, 269, 202, 32998, 30123, 33263, 32380, 388, 9724, 37985, 1172, 397, 35860, 28999, 20057, 3092, 37033, 38084, 3750, 2273, 4080, 254, 29755, 3665, 37150, 39597, 82, 12403, 11887, 267, 15771, 13119, 11135, 34745, 253, 29787, 3782, 2321, 1505, 4447, 5630, 12413, 1254, 35507, 12445, 38349, 9465, 1472, 38000, 12893, 4898, 35073, 8142, 21185, 14349, 249, 532, 19111, 569, 597, 34504, 1729, 15809, 23225, 22319, 10521, 949, 17909, 1366, 35678, 35075, 8310, 543, 890, 33190, 22097, 2035, 6377, 4163, 2143, 6007, 400, 20, 2890, 6351, 6702, 3014, 33221, 9088, 25517, 187, 14175, 1695, 27947, 35120, 10012, 4985, 429, 3572, 10029, 38250, 16916, 7062, 8408, 24921, 26123, 30745, 13767, 33899, 790, 22, 9926, 2158, 658, 780, 262

In [6]:
# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = reviews['profile'].values
movie_list = reviews['anime_uid'].values
for j in range(len(reviews)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
reviews['profile'] = user_list
reviews['anime_uid'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
train_index = np.random.random(len(reviews)) <= 0.7
train_df = reviews[train_index]
test_df = reviews[~train_index]

# generate train_mat and test_mat
num_user = len(reviews['profile'].unique())
num_items = len(reviews['anime_uid'].unique())

train_mat = coo_matrix((train_df['score'].values, (train_df['profile'].values, train_df['anime_uid'].values)), shape=(num_user, num_items)).astype(float).toarray()
test_mat = coo_matrix((test_df['score'].values, (test_df['profile'].values, test_df['anime_uid'].values)), shape=(num_user, num_items)).astype(float).toarray()
train_data = torch.FloatTensor(train_mat)
test_data = torch.FloatTensor(test_mat)

In [12]:
movie_old2new_id_dict.keys()

dict_keys([34096, 34599, 28891, 2904, 4181, 16664, 4672, 5114, 30276, 1253, 1425, 7588, 20767, 22673, 6862, 451, 9563, 30364, 33280, 36369, 27633, 8609, 36214, 5680, 31637, 552, 38145, 34944, 1689, 35363, 32863, 322, 38958, 7645, 468, 97, 39940, 4472, 19319, 1571, 38770, 5005, 18397, 269, 202, 32998, 30123, 33263, 32380, 388, 9724, 37985, 1172, 397, 35860, 28999, 20057, 3092, 37033, 38084, 3750, 2273, 4080, 254, 29755, 3665, 37150, 39597, 82, 12403, 11887, 267, 15771, 13119, 11135, 34745, 253, 29787, 3782, 2321, 1505, 4447, 5630, 12413, 1254, 35507, 12445, 38349, 9465, 1472, 38000, 12893, 4898, 35073, 8142, 21185, 14349, 249, 532, 19111, 569, 597, 34504, 1729, 15809, 23225, 22319, 10521, 949, 17909, 1366, 35678, 35075, 8310, 543, 890, 33190, 22097, 2035, 6377, 4163, 2143, 6007, 400, 20, 2890, 6351, 6702, 3014, 33221, 9088, 25517, 187, 14175, 1695, 27947, 35120, 10012, 4985, 429, 3572, 10029, 38250, 16916, 7062, 8408, 24921, 26123, 30745, 13767, 33899, 790, 22, 9926, 2158, 658, 780, 262

In [8]:
def predict_rating(user_index, item_index, ratings, item_similarity):

    numerator = 0
    denominator = 0
    for i in range(ratings.shape[1]):
        if i != item_index and ratings[user_index][i] != 0:
            numerator += ratings[user_index][i] * item_similarity[item_index][i]
            denominator += item_similarity[item_index][i]
    if denominator == 0:
        return 0
    else:
        return numerator / denominator

In [234]:
"""
#Creating a dictionary for item similarity
new_item_dict_from_old = {}
rev_new_item_dict_from_old = {}

key_vit_50_dict = [x for x in vit_50_dict.keys()]
failed_list = []
j=0
for x in key_vit_50_dict:
  new_item_dict_from_old[x] = j
  rev_new_item_dict_from_old[j] = x
  j=j+1
"""

'\n#Creating a dictionary for item similarity\nnew_item_dict_from_old = {}\nrev_new_item_dict_from_old = {}\n\nkey_vit_50_dict = [x for x in vit_50_dict.keys()]\nfailed_list = []\nj=0\nfor x in key_vit_50_dict:\n  new_item_dict_from_old[x] = j\n  rev_new_item_dict_from_old[j] = x\n  j=j+1\n'

In [10]:
import numpy as np
def cosine_similarity(a, b):
    similarity = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return similarity


In [28]:
key_vit_50_dict = [x for x in vit_50_dict.keys()]
simm_item_ij = np.zeros((len(key_vit_50_dict), len(key_vit_50_dict)))
for i in key_vit_50_dict:
  for j in key_vit_50_dict:
    embd_i = vit_50_dict[i].reshape(-1)
    embd_j = vit_50_dict[j].reshape(-1)
    item_i_idx = movie_old2new_id_dict[i]
    item_j_idx = movie_old2new_id_dict[j]
    simm_item_ij[item_i_idx][item_j_idx] = cosine_similarity(embd_i, embd_j)



In [107]:
simm_item_ij

array([[0.99999994, 0.73196971, 0.6469667 , ..., 0.95363891, 0.95695341,
        0.69157743],
       [0.73196971, 0.99999994, 0.98086721, ..., 0.66420573, 0.72470284,
        0.99740493],
       [0.6469667 , 0.98086721, 1.00000012, ..., 0.62096447, 0.68534744,
        0.99146509],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
cnt = test_data[0]*test_data[1]
sum_diff = 0
for user_id in range(test_data.shape[0]):
  for item_id in range(test_data.shape[1]):
    predicted_rating = predict_rating(user_id, item_id, train_data, simm_item_ij)
    actual_rating = test_data[user_id][item_id]
    diff = (actual_rating - predicted_rating)*(actual_rating - predicted_rating)
    sum_diff = sum_diff + diff 

rmse = sum_diff/cnt 
print("RMSE = " + str(rmse))
